In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
# Calculate silhouette_score
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
#Read files
trainfile = r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 5/2/SouthGermanCredit.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
#testfile = r'/gdrive/My Drive/Asim Roy/Datasets/ECG-TEST.csv'
#testData = pd.read_csv(testfile)  #creates a dataframe

print(trainData.shape)
#print(testData.shape)


(1000, 21)


In [ ]:
#Extract training and test data
y_train = trainData["kredit"]
X_train = trainData.drop(["kredit"], axis=1) #extracting training data without the target column
#y_test = testData["TARGET"]
#X_test = testData.drop(["TARGET"], axis=1) #extracting training data without the target column
print(X_train.head())
#print(X_test.shape)


   laufkont  laufzeit  moral  verw  ...  beruf  pers  telef  gastarb
0         1        18      4     2  ...      3     2      1        2
1         1         9      4     0  ...      3     1      1        2
2         2        12      2     9  ...      2     2      1        2
3         1        12      4     0  ...      2     1      1        1
4         1        12      4     0  ...      2     2      1        1

[5 rows x 20 columns]


In [ ]:
hpt = {"init":"random", "algorithm":"full"}

In [ ]:
hpt = {"init":"random", "algorithm":"full"}
for i in range (2,15):
  kmeans=KMeans(n_clusters=i,**hpt)
  kmeans.fit(X_train)
  score=silhouette_score(X_train,kmeans.labels_)
  print('k=',i,'silhouette score is',score)

k= 2 silhouette score is 0.7216367721912854
k= 3 silhouette score is 0.6475684539923428
k= 4 silhouette score is 0.6117259380414384
k= 5 silhouette score is 0.5776798542438559
k= 6 silhouette score is 0.5764133981584135
k= 7 silhouette score is 0.5680435284455725
k= 8 silhouette score is 0.5347295412472617
k= 9 silhouette score is 0.5508563032237214
k= 10 silhouette score is 0.5703859485779206
k= 11 silhouette score is 0.5542838332153113
k= 12 silhouette score is 0.5574628383694804
k= 13 silhouette score is 0.5340016061995521
k= 14 silhouette score is 0.5653531703957387


In [ ]:
#K-Means Clustering ========================================================================


kmeans = KMeans(n_clusters=2,**hpt)
kmeans.fit(X_train)
kmeans.labels_

#Add Cluster Number to each datapoint + save file
df_kmeans= X_train
kmeans_predict_train=kmeans.predict(df_kmeans)
df_kmeans['Cluster Number']=kmeans_predict_train
print(df_kmeans.shape)
#df_kmeans.head()
#df_kmeans.dtypes
df_kmeans= pd.concat([df_kmeans, y_train],axis=1)

export_csv = df_kmeans.to_csv(r'/gdrive/My Drive/ASU_MSBA/CIS_508/Assignment 5/2/kmeans_SouthGermanCredit.csv')

(1000, 21)


In [ ]:
# use target cluster number
data2=df_kmeans.loc[:,["kredit", "Cluster Number"]]
NG=data2.pivot_table(index='Cluster Number', columns= 'kredit', aggfunc=len)
NG['Total']= NG[0]+NG[1]
NG['Bad Credit%'] =(NG[0]/NG.Total)*100
NG['Good Credit %']= (NG[1]/NG.Total)*100
print(NG)

kredit            0    1  Total  Bad Credit%  Good Credit %
Cluster Number                                             
0               228  599    827    27.569528      72.430472
1                72  101    173    41.618497      58.381503


In [ ]:
# Mapping clusters
y_pred = 1 - kmeans_predict_train
# accuracy score
accuracy_score(y_train,y_pred)

0.671

In [ ]:
centroids = kmeans.cluster_centers_
X_train1 = trainData.drop(["kredit"], axis=1)
df_centroids = pd.DataFrame(centroids, columns=X_train1.columns)
df_centroids.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,wohnzeit,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb
0,2.596131,17.970979,2.551391,2.766626,2193.293833,2.049577,3.380895,3.074970,2.677146,1.151149,2.835550,2.229746,35.287787,2.694075,1.896010,1.395405,2.835550,1.847642,1.354293,1.960097
1,2.485549,34.919075,2.514451,3.121387,8424.242775,2.369942,3.398844,2.485549,2.705202,1.115607,2.890173,2.971098,36.757225,2.583815,2.080925,1.462428,3.231214,1.832370,1.641618,1.976879
